In [1]:
#Importing the required Libraries
import requests as r
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time

In [2]:
#Key Takeaways
#1-It took me time to find the main key that was holding all the information for our apartment search
#2-1 API
#3-As part of making the web scrapping interactive with the client, I have asked for 10 user inputs (Filters)
#4-As soon as I get the request URL, a <Response 400> message appears -> Had to include requested headers to grant myself access to scrape
#5-As part of the requested inputs (# of pages to be scraped), I faced multiple issues in the outputed DF, where when the user inputed >1 page to be scraped, it displayed only 1 full DF (for 1 page) and the rest NaN
#6-Remove Federated Search ID and Source keys from URL
#7-Incognito Mode: Not providing all personalized cookies, only necessary ones
#8-Removed cookies for the input to display unique concatenated DF

In [3]:
#For looping across pages and keeping cookies -> Repeated DF
#When you remove the cookies, it starts to output a correct DF, however not matching the ones on Airbnb
def main():
    output_final=pd.DataFrame()
    df5=pd.DataFrame()
    outputs=[]
    city=input('What city would you like to search? ')
    country=input('What country would you like to search? ')
    checkin=input('Check in date (YYYY-MM-DD) ')
    checkout=input('Check out date (YYYY-MM-DD) ')
    adults=input('Number of adults (max:16) ')
    children=input('Number of Children (max:5) ')
    infants=input('Number of Infants (max:5) ')
    minprice=input('Minimum Price ')
    maxprice=input('Maximum Price ')
    pages=int(input('How many pages do you want to display (20 stays in each page)? '))
    
    for i in range(1,pages+1):
        url=f'https://www.airbnb.com/api/v3?locale=en&currency=EUR&operationName=ExploreSearch&variables=%7B%22request%22%3A%7B%22metadataOnly%22%3Afalse%2C%22version%22%3A%221.7.5%22%2C%22itemsPerGrid%22%3A20%2C%22tabId%22%3A%22home_tab%22%2C%22refinementPaths%22%3A%5B%22%2Fhomes%22%5D%2C%22placeId%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22searchType%22%3A%22pagination%22%2C%22mapToggle%22%3Afalse%2C%22checkin%22%3A%22{checkin}%22%2C%22checkout%22%3A%22{checkout}%22%2C%22adults%22%3A{adults}%2C%22children%22%3A{children}%2C%22infants%22%3A{infants}%2C%22priceMin%22%3A{minprice}%2C%22priceMax%22%3A{maxprice}%2C%22sectionOffset%22%3A4%2C%22itemsOffset%22%3A{(pages-1)*20}%2C%22query%22%3A%22{city}%2C%20{country}%22%2C%22cdnCacheSafe%22%3Afalse%2C%22simpleSearchTreatment%22%3A%22simple_search_only%22%2C%22screenSize%22%3A%22large%22%7D%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2246a11d18adaed400e9afc6189e2d38a11cc61bb9cc33f3c339e803c9fc0666ec%22%7D%7D'
        headers=f"""accept: */*
        accept-encoding: gzip, deflate, br
        accept-language: en-US,en;q=0.9,ar;q=0.8
        cache-control: no-cache
        content-type: application/json
        pragma: no-cache
        user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
        x-airbnb-api-key: d306zoyjsyarp7ifhu67rjxn52tv0t20
        x-airbnb-graphql-platform: web
        x-airbnb-graphql-platform-client: apollo-niobe
        x-csrf-token: V4$.airbnb.com$23WaWePkaEg$hAiUsTN8iJzF_JJb9vViRXO9FBtUWTJcPHmjb9AZrIY=
        x-csrf-without-token: 1"""
    
        headers=dict([j.strip().split(': ') for j in headers.split('\n')])
        output=r.get(url, headers=headers).json()
        lst=pd.json_normalize(output['data']['dora']['exploreV3']['sections'][0]['items'])
        output_final=pd.concat([output_final,lst],axis=0)   
        
        x=output['data']['dora']['exploreV3']['sections'][0]['items']
        y=[[x[i]['listing']['pictureUrls']] for i in range(len(x))]
        df2=pd.DataFrame(y,columns=['Pictures URLS'])
        z=[x[i]['pricingQuote']['priceString'] for i in range(len(x))]
        df3=pd.DataFrame(z,columns=['Price'])
        df4=pd.concat([df2,df3], axis=1)
        df5=pd.concat([df5,df4],axis=0)
        
        
        
    return pd.concat([output_final,df5],axis=1).reset_index()

df_final=main()
df_final=df_final[['listing.name','listing.localizedNeighborhood','listing.publicAddress','listing.spaceType','listing.bathrooms','listing.beds','listing.bedrooms','listing.previewAmenities','listing.cancellationPolicyTitle','listing.avgRating','Pictures URLS','Price']]
df_final=df_final.rename(columns={'listing.contextualPictures':'Pictures','listing.name':'Description', 'listing.bathrooms':'Bathroom', 'listing.beds':'Beds','listing.bedrooms':'Room','listing.previewAmenities':'Ameneties','listing.cancellationPolicyTitle':'Cancelation Policy','listing.localizedNeighborhood':'Neighborhood','listing.spaceType':'Space Type', 'listing.publicAddress': 'Address', 'listing.avgRating':'Rating'})
   
    

What city would you like to search? Paris
What country would you like to search? France
Check in date (YYYY-MM-DD) 2020-04-20
Check out date (YYYY-MM-DD) 2020-04-25
Number of adults (max:16) 1
Number of Children (max:5) 1
Number of Infants (max:5) 1
Minimum Price 100
Maximum Price 400
How many pages do you want to display (20 stays in each page)? 2


In [4]:
df_final

,Description,Neighborhood,Address,Space Type,Bathroom,Beds,Room,Ameneties,Cancelation Policy,Rating,Pictures URLS,Price
0,"Beauquartier Madeleine, Le Second Beau Studio",VIII Arrondissement,"Paris, Île-de-France, France",Entire apartment,1.0,1,0,"Wifi,Kitchen,Heating,Washer",None,4.93,[https://a0.muscache.com/4ea/air/v2/pictures/7...,€117
1,Superb flat 200m from the Luxembourg Gardens,Montparnasse,"Paris, Île-de-France, France",Entire apartment,1.0,5,2,"Wifi,Kitchen,Heating,Washer",None,4.96,[https://a0.muscache.com/im/pictures/73e1de65-...,€180
2,New Appartement 3 Pieces Kléber / Arc de Triomphe,XVI Arrondissement,"Paris, Île-de-France, France",Entire apartment,1.5,4,2,"Wifi,Kitchen,Heating,Washer",None,4.91,[https://a0.muscache.com/im/pictures/6f0b229c-...,€220
3,Lovely Loft Saint-Germain des Pres,Saint-Germain-des-Prés - Odéon,"Paris, Ile-de-France, France",Entire apartment,1.5,1,1,"Wifi,Kitchen,Heating,Washer",None,4.63,[https://a0.muscache.com/im/pictures/ef3544b7-...,€114
4,[110m2] Eiffel Gorgeous Apartment,Tour Eiffel - Champ de Mars,"Paris, Île-de-France, France",Entire apartment,2.0,5,3,"Wifi,Kitchen,Heating,Washer",None,4.82,[https://a0.muscache.com/im/pictures/63326125/...,€255
5,The Residence Luxury 2 Bedrooms Le Marais I,Le Marais,"Paris, Île-de-France, France",Entire apartment,1.0,3,2,"Wifi,Kitchen,Heating,Washer",None,4.87,[https://a0.muscache.com/im/pictures/d6095719-...,€99
6,Luxury 3 bedroom 2 bathroom Palace Apartment - AC,II Arrondissement,"Paris, Île-de-France, France",Entire apartment,2.5,4,3,"Wifi,Kitchen,Heating,Washer",None,4.86,[https://a0.muscache.com/im/pictures/b627955f-...,€140
7,"Bright, Updated Flat Near Notre Dame",Saint-Germain-des-Prés - Odéon,"Paris, Île-de-France, France",Entire apartment,1.0,2,1,"Wifi,Kitchen,Heating,Washer",None,4.90,[https://a0.muscache.com/4ea/air/v2/pictures/8...,€214
8,Appartement au pied de Saint-Lazare / Opera,VIII Arrondissement,"Paris, Île-de-France, France",Entire apartment,1.0,2,1,"Wifi,Kitchen,Heating",None,5.00,[https://a0.muscache.com/im/pictures/af4c43c6-...,€116
9,"AMAZING FLAT STYLE LOFT (old workshop), LOUVRE",II Arrondissement,"Paris, Île-de-France, France",Entire apartment,1.0,2,1,"Wifi,Kitchen,Heating,Washer",None,4.91,[https://a0.muscache.com/im/pictures/cacdcd19-...,€153
